In [1]:
from getpass import getpass

import psycopg2
from psycopg2.extras import execute_values

# 1. Connecting to the Postgres Server

- It's best to create a *temporary/dummy user* to work with *just for this session*. You can do so by running the following command in a terminal:

    ```bash
    createuser psycopg2_test_user --createdb -P
    ```

- It's also recommended to create a *new database* to work in:

    ```bash
    createdb psycopg2_test_db
    ```

**NOTE:** All created objects will be removed in the end.

In [2]:
# Establish a connection to the database
conn = psycopg2.connect(
    dbname="psycopg2_test_db",
    user="psycopg2_test_user",
    password=getpass(prompt="User password: "),
    host="localhost",
    port="5432"
)

User password: ········


# 2. Basic Database Operations

## 2.1 Creating tables

In [3]:
with conn:
    with conn.cursor() as cursor:
        # Create a table of UTF-8 characters
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS utf8_chars (
                code_point   integer,
                character    text
            );
            """
        )

        # Create a table to hold sample export data
        cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS exports (
                type        text NOT NULL,
                commodity   text NOT NULL,
                mass_in_kg  numeric NOT NULL,
                price       money NOT NULL,
                date        timestamp NOT NULL
            );
            """
        )

## 2.2 Inserting data

In [4]:
with conn:
    with conn.cursor() as cursor:
        # Insert one row into the "utf8_chars" table
        cursor.execute("INSERT INTO utf8_chars VALUES (%s, %s)", (1, chr(1)))

        # Insert multiple rows (very inefficient)
        cursor.executemany(
            "INSERT INTO utf8_chars VALUES (%s, %s)",
            [(code_point, chr(code_point)) for code_point in range(2, 50)]
        )
        # Insert multiple rows efficiently
        execute_values(
            cur=cursor,
            sql="INSERT INTO utf8_chars VALUES %s",
            argslist=[(code_point, chr(code_point)) for code_point in range(50, 128)]
        )

        # Import data from a CSV file into the "exports" table
        with open("exports_data.csv") as exports_file:
            header = exports_file.readline()
            cursor.copy_from(file=exports_file, table="exports", sep=",", null="")

## 2.3 Fetching data

In [5]:
with conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT * FROM utf8_chars;")
        print(
            "fetchone(): ",
            cursor.fetchone(),  # Fetch the next row of a query result set
            "\n\nfetchmany(10): ",
            cursor.fetchmany(10),  # Fetch the next `size` rows of a query result
            "\n\nfetchall(): ",
            cursor.fetchall()  # Fetch all (remaining) rows of a query result
        )
        
        cursor.execute(
            """
            SELECT type, commodity, price
              FROM exports
              WHERE (price > %s)
              ORDER BY price DESC;
            """,
            (("1000000000"),)
        )
        print("\n\nTransactions worth billions: \n", *cursor.fetchall(), sep="\n")

fetchone():  (1, '\x01') 

fetchmany(10):  [(2, '\x02'), (3, '\x03'), (4, '\x04'), (5, '\x05'), (6, '\x06'), (7, '\x07'), (8, '\x08'), (9, '\t'), (10, '\n'), (11, '\x0b')] 

fetchall():  [(12, '\x0c'), (13, '\r'), (14, '\x0e'), (15, '\x0f'), (16, '\x10'), (17, '\x11'), (18, '\x12'), (19, '\x13'), (20, '\x14'), (21, '\x15'), (22, '\x16'), (23, '\x17'), (24, '\x18'), (25, '\x19'), (26, '\x1a'), (27, '\x1b'), (28, '\x1c'), (29, '\x1d'), (30, '\x1e'), (31, '\x1f'), (32, ' '), (33, '!'), (34, '"'), (35, '#'), (36, '$'), (37, '%'), (38, '&'), (39, "'"), (40, '('), (41, ')'), (42, '*'), (43, '+'), (44, ','), (45, '-'), (46, '.'), (47, '/'), (48, '0'), (49, '1'), (50, '2'), (51, '3'), (52, '4'), (53, '5'), (54, '6'), (55, '7'), (56, '8'), (57, '9'), (58, ':'), (59, ';'), (60, '<'), (61, '='), (62, '>'), (63, '?'), (64, '@'), (65, 'A'), (66, 'B'), (67, 'C'), (68, 'D'), (69, 'E'), (70, 'F'), (71, 'G'), (72, 'H'), (73, 'I'), (74, 'J'), (75, 'K'), (76, 'L'), (77, 'M'), (78, 'N'), (79, 'O'), (80, '

## 2.4 Deleting data

In [6]:
with conn:
    with conn.cursor() as cursor:
        cursor.execute(
            """
            DELETE FROM utf8_chars
              WHERE (code_point < %(lower)s OR code_point >= %(upper)s);
            """,
            {"lower": 65, "upper": 91}
        )
        cursor.execute("SELECT * FROM utf8_chars LIMIT 5;")
        print(cursor.fetchall())

[(65, 'A'), (66, 'B'), (67, 'C'), (68, 'D'), (69, 'E')]


## 2.5 Basic operations and aggregations

In [7]:
with conn:
    with conn.cursor() as cursor:
        cursor.execute(
            """
            SELECT commodity, sum(mass_in_kg) AS total_mass
              FROM exports
              GROUP BY commodity
              ORDER BY total_mass DESC
              LIMIT 5;
            """
        )
        print("Most traded commodities by mass: \n", *cursor.fetchall(), sep="\n")

Most traded commodities by mass: 

('Roses', Decimal('105164149.83'))
('Avocado', Decimal('31225859.63'))
('Mixed vegetables', Decimal('28129937.97'))
('Fine beans', Decimal('23090599.07'))
('Mango', Decimal('14047648.26'))


In [8]:
with conn:
    with conn.cursor() as cursor:
        cursor.execute(
            """
            SELECT type, commodity, sum( price::numeric / mass_in_kg) AS price_per_kg
              FROM exports
              WHERE (mass_in_kg > %s)
              GROUP BY type, commodity
              ORDER BY price_per_kg DESC
              LIMIT 5;
            """,
            ((0),)
        )
        print("Top 5 price-per-kg: \n", *cursor.fetchall(), sep="\n")

Top 5 price-per-kg: 

('Cut-flowers', 'Geranium cuttings', Decimal('2304661.6148465367153686'))
('Cut-flowers', 'Asters', Decimal('137779.5771915048831234'))
('Cut-flowers', 'Cuttings', Decimal('92254.5111934085350602'))
('Cut-flowers', 'Phlox', Decimal('84835.6615179044565443'))
('Cut-flowers', 'Chrysanthemums cuttings', Decimal('56395.0385073169622280'))


In [9]:
with conn:
    with conn.cursor() as cursor:
        cursor.execute(
            """
            SELECT type, max(mass_in_kg), min(mass_in_kg), round(avg(mass_in_kg), 4)
              FROM exports
              GROUP BY type;
            """
        )
        print(
            "Maximum, minimum and average mass by type: \n", *cursor.fetchall(), sep='\n'
        )

Maximum, minimum and average mass by type: 

('Fruits', Decimal('5302741.82'), Decimal('0.0'), Decimal('296450.9591'))
('Cut-flowers', Decimal('11049185.48'), Decimal('0.0'), Decimal('127942.7788'))
('Vegetables', Decimal('3565356.17'), Decimal('0.0'), Decimal('102592.8793'))


# 3. Clean Up (Removing objects)

In [10]:
with conn:
    with conn.cursor() as cur:
        # Delete the created tables
        cur.execute("DROP TABLE utf8_chars;")
        cur.execute("DROP TABLE exports;")
        
        # Remove everything owned by our dummy user 
        cur.execute("DROP OWNED BY psycopg2_test_user;")

# Close the connection to database
conn.close()

In [11]:
%%bash
# Delete the "psycopg2_test_db" database
dropdb psycopg2_test_db

# Delete the dummy user
dropuser psycopg2_test_user